In [2]:
import pandas as pd
import sqlalchemy
dados = pd.read_parquet('br_base\Restaurantes e similares.gzip')
dados.columns

Index(['CNPJ', 'RAZAO_SOCIAL', 'NOME_FANTASIA', 'RUA', 'NUMERO', 'COMPLEMENTO',
       'BAIRRO', 'CIDADE', 'ESTADO', 'CEP', 'LATITUDE', 'LONGITUDE',
       'TELEFONE1', 'SITE', 'CNAE_DESCRICAO', 'HORARIO_FUNCIONAMENTO',
       'INSTAGRAM', 'FACEBOOK', 'OPCOES_DE_SERVICO', 'SITUACAO_CADASTRAL',
       'DATA_SITUACAO_CADASTRAL'],
      dtype='object')

In [30]:
import duckdb

# Criando um objeto duckdb
duck = duckdb.connect()

# consulta
query = """
SELECT DATA_SITUACAO_CADASTRAL, 
COUNT(*) as TOTAL
FROM dados 
WHERE CNAE_DESCRICAO = 'RESTAURANTES E SIMILARES'
GROUP BY DATA_SITUACAO_CADASTRAL;
"""

# Exemplo de consulta SQL usando duckdb
resultado = duck.query(query).df()

resultado

,DATA_SITUACAO_CADASTRAL,TOTAL
0,20221123,3532
1,20200318,169
2,20200422,112
3,20230124,329
4,20200707,136
...,...,...
6857,20101006,22
6858,20120405,6
6859,20070514,9
6860,20051222,9


In [43]:
ultima_alteracao = duck.query("""
SELECT DATA_SITUACAO_CADASTRAL 
FROM dados
ORDER BY DATA_SITUACAO_CADASTRAL DESC
LIMIT 1;
""").fetchone()[0]

ultima_alteracao

'20230211'

In [3]:
print(dados['CNAE_DESCRICAO'].unique())

['RESTAURANTES E SIMILARES']
